# ADS‑B Flight Segmentation and Visualization
This notebook loads ADS‑B logs via SFTP, segments flights by time gaps, calculates metrics, and displays an interactive map with date and
irregularity filters to display the anomalous flight paths, and data download option based on the filters selected.

## 1. Imports & Constants

In [1]:
import os
import sys
import io
import zipfile
import json
import ast
import math
import base64
import yaml
import paramiko
import pandas as pd
from datetime import datetime, date
from IPython.display import display
from ipywidgets import DatePicker, FloatSlider, Button, HTML, VBox
from ipyleaflet import Map, GeoJSON, Popup, WidgetControl, ControlException


## 2. Configuration

In [2]:
# Read a YAML configuration file and return its contents as a dictionary.
def read_config(path: str) -> dict:
    if not os.path.exists(path):
        sys.exit(
            "ERROR: ‘config.yml’ not found.\n"
            "Please copy `config.example.yml` → `config.yml` and fill in your credentials."
        )
    with open(path) as f:
        return yaml.safe_load(f)

# Load the configuration from the file
config = read_config('config.yml')

# Extract the base server path from the config and trim any extra whitespace
BASE_SERVER_PATH = config['site']['base_server_path'].strip()

# Build the full path to the log directory by joining base path and the log path setting
LOG_PATH = os.path.join(
    BASE_SERVER_PATH,
    config['site']['log_path'].strip()
)

# Print out the resolved log path so you can verify it when the script runs
# print(f'Log path: {LOG_PATH}')


## 3. SFTP Connection Helpers

In [3]:
def connect_to_server(cfg: dict) -> paramiko.SSHClient:
    """
    Establish an SSH connection to a remote server using Paramiko.

    Parameters:
        cfg (dict): Configuration dictionary with a 'site' key containing:
            - hostname (str): Server address or IP.
            - port (int): SSH port (usually 22).
            - username (str): SSH login user.
            - password (str, optional): SSH password (if not using key‑based auth).

    Returns:
        paramiko.SSHClient: An active SSHClient connected to the target host.
    """
    ssh = paramiko.SSHClient()  
    ssh.load_system_host_keys()  
    # Automatically add unknown host keys (use with care in production)
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    site = cfg['site']
    ssh.connect(
        hostname=site['hostname'],
        port=site['port'],
        username=site['username'],
        password=site.get('password')  # None if not provided
    )
    return ssh

def get_sftp_client(cfg: dict) -> tuple:
    """
    Open an SFTP session over an existing SSH connection.

    Parameters:
        cfg (dict): Same config dict passed to `connect_to_server`.

    Returns:
        tuple:
            - paramiko.SSHClient: The underlying SSH client.
            - paramiko.SFTPClient: An SFTP client for file operations.
    """
    ssh = connect_to_server(cfg)
    sftp = ssh.open_sftp()  # Start SFTP subsystem over SSH
    return ssh, sftp


## 4. Data Processing Functions

In [4]:
def build_remote_zip_path(log_date: date, base_path: str) -> str:
    """
    Construct the full path to a remote ZIP log file for a given date.

    Parameters:
        log_date (date): The date of the log you want, e.g., datetime.date(2022, 10, 4).
        base_path (str): The directory on the server where log ZIPs are stored.

    Returns:
        str: A filepath like "/remote/logs/adsblog_ny0.txt.2022100400.zip".
    """
    # Format the date as YYYYMMDD and append "00.zip" to match the naming convention
    fname = f"adsblog_ny0.txt.{log_date.strftime('%Y%m%d')}00.zip"
    # Join the base directory and filename into a single, OS‑aware path string
    return os.path.join(base_path, fname)

In [5]:
# Columns to remove once we’ve flattened the ADS‑B payload into tabular form
COLUMNS_TO_DROP = [
    'nav_altitude_fms', 'seen', 'messages', 'tisb', 'mlat', 'gva',
    'sil_type', 'sil', 'nac_v', 'nac_p', 'nic_baro', 'version', 'nic',
    'rc', 'nav_modes', 'squawk', 'track_rate', 'roll', 'tas', 'ias',
    'mach', 'mag_heading', 'geom_rate', 'nav_heading', 'baro_rate',
    'nav_altitude_mcp', 'nav_qnh', 'seen_pos', 'sda', 'category'
]

def load_and_clean_adsb_zip_json(sftp, remote_zip_path: str) -> pd.DataFrame:
    """
    Download a zipped ADS‑B log file over SFTP, extract JSON lines,
    filter for new ADS‑B messages, and return a cleaned DataFrame.

    Parameters:
        sftp (paramiko.SFTPClient): Open SFTP session on the remote server.
        remote_zip_path (str): Full path to the .zip archive on the server.

    Returns:
        pd.DataFrame: Table of ADS‑B payload fields + 'date', minus unused columns.
    """
    # 1. Read the raw ZIP bytes from the remote file
    with sftp.open(remote_zip_path, 'rb') as rf:
        raw = rf.read()

    # 2. Load the ZIP archive from bytes
    with zipfile.ZipFile(io.BytesIO(raw)) as zf:
        # List all entries that are not directories
        all_files = [n for n in zf.namelist() if not n.endswith('/')]
        # Prefer the .txt file, or fall back to the first member
        member = next((n for n in all_files if n.lower().endswith('.txt')), all_files[0])

        rows = []
        # 3. Stream each line from the chosen member file
        with zf.open(member) as f:
            for raw_line in f:
                try:
                    # Decode and parse the JSON object
                    obj = json.loads(raw_line.decode('utf-8').strip())
                    # Only keep records of type 'new_adsb'
                    if obj.get('type') == 'new_adsb':
                        p = obj['payload']
                        p['date'] = obj['dt']  # Preserve the timestamp
                        rows.append(p)
                except:
                    # Skip any malformed lines silently
                    continue

    # 4. Build a DataFrame and drop all columns we don't need
    df = pd.DataFrame(rows)
    return df.drop(columns=COLUMNS_TO_DROP, errors='ignore')

In [6]:
def clean_adsb_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert the 'date' column to real datetime objects and
    sort the DataFrame by aircraft identifier and timestamp.

    Parameters:
        df (pd.DataFrame): Raw ADS‑B DataFrame with a 'date' column.

    Returns:
        pd.DataFrame: Cleaned DataFrame with:
            - 'date' as datetime dtype (invalid parsing becomes NaT)
            - Rows sorted by ['hex', 'date']
            - Continuous integer index (ignore_index=True)
    """
    # Ensure 'date' is a pandas datetime, coercing invalid strings to NaT
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    # Sort first by the aircraft 'hex' code, then by timestamp
    # Reset the index so it's contiguous after sorting
    return df.sort_values(['hex', 'date'], ignore_index=True)

In [7]:
def split_by_time_gap(group: pd.DataFrame, max_gap_minutes: int = 10) -> pd.DataFrame:
    """
    Split a sequence of ADS‑B messages for one flight into segments whenever
    there is a gap larger than `max_gap_minutes` between consecutive points.

    Parameters:
        group (pd.DataFrame): DataFrame for a single flight (e.g., grouped by 'hex' & 'flight'),
                              containing at least a 'date' column of dtype datetime64.
        max_gap_minutes (int): Threshold (in minutes) above which a new segment starts.

    Returns:
        pd.DataFrame: The same rows, with two new columns:
            - 'time_diff': gap to the previous point in minutes (NaN for the first row)
            - 'segment': integer segment ID (starting at 0), incremented each time gap > threshold
    """
    # 1. Ensure rows are in time order before computing differences
    group = group.sort_values('date')

    # 2. Compute time difference (in minutes) between each timestamp and its predecessor
    group['time_diff'] = group['date'].diff().dt.total_seconds() / 60

    # 3. Create a segment counter: start a new segment whenever time_diff > max_gap_minutes
    #    .cumsum() will increment the segment ID each time the condition is True (treated as 1)
    group['segment'] = (group['time_diff'] > max_gap_minutes).cumsum()

    return group


In [8]:
def segment_adsb(df: pd.DataFrame, max_gap: int = 10) -> pd.DataFrame:
    """
    Divide the full ADS‑B DataFrame into time‑continuous segments for each flight.

    Parameters:
        df (pd.DataFrame): Cleaned ADS‑B DataFrame containing at least
                           'hex', 'flight', and 'date' columns.
        max_gap (int): Maximum gap in minutes before a new segment starts.

    Returns:
        pd.DataFrame: Original DataFrame plus 'time_diff' and 'segment' columns,
                      with a fresh integer index.
    """
    # 1. Group by aircraft ('hex') and flight ID, without inserting group keys into the index
    # 2. Apply split_by_time_gap to each group to compute time differences and segment IDs
    # 3. Reset the combined index so rows are numbered 0..n-1
    return (
        df
        .groupby(['hex', 'flight'], group_keys=False)
        .apply(lambda grp: split_by_time_gap(grp, max_gap))
        .reset_index(drop=True)
    )

In [9]:
def aggregate_flight_segments(g: pd.DataFrame) -> dict:
    """
    Summarize a single flight’s segment into start/end times, total duration, and the path.

    Parameters:
        g (pd.DataFrame): A DataFrame for one flight segment, containing at least:
            - 'date' (datetime64): Timestamps of each position report.
            - 'lat', 'lon' (float): Latitude and longitude of each report.

    Returns:
        dict: A summary with keys:
            - 'start_time' (Timestamp): Earliest timestamp in this segment.
            - 'end_time' (Timestamp): Latest timestamp in this segment.
            - 'duration' (float): Total flight time in minutes.
            - 'coordinates' (List[tuple]): Ordered list of (lat, lon) pairs.
    """
    # 1. Drop any records missing coordinates, then sort chronologically
    pts = g.dropna(subset=['lat', 'lon']).sort_values('date')

    # 2. Build a list of (latitude, longitude) tuples in time order
    coords = list(zip(pts['lat'], pts['lon']))

    # 3. Compute summary metrics
    start = g['date'].min()
    end   = g['date'].max()
    # Duration in minutes between first and last timestamps
    duration = (end - start).total_seconds() / 60

    return {
        'start_time':  start,
        'end_time':    end,
        'duration':    duration,
        'coordinates': coords
    }

In [10]:
def aggregate_segments(df: pd.DataFrame) -> pd.DataFrame:
    """
    Roll up each flight segment into a summary record using `aggregate_flight_segments`.

    Parameters:
        df (pd.DataFrame): ADS‑B DataFrame that has been segmented, containing columns:
            - 'hex', 'flight', 'segment' for grouping
            - 'date', 'lat', 'lon' (and any other payload fields used by the aggregator)

    Returns:
        pd.DataFrame: One row per (hex, flight, segment) group with columns:
            - 'hex', 'flight', 'segment'
            - 'start_time', 'end_time', 'duration', 'coordinates'
    """
    # 1. Group by aircraft code, flight ID, and computed segment
    # 2. For each group, call aggregate_flight_segments to get a dict of summary values
    # 3. Convert each dict into a pandas Series (so keys become columns)
    # 4. Reset the index so that 'hex', 'flight', 'segment' become normal columns again
    return (
        df
        .groupby(['hex', 'flight', 'segment'])
        .apply(lambda g: pd.Series(aggregate_flight_segments(g)))
        .reset_index()
    )

In [11]:
def haversine(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """
    Calculate the great-circle distance between two points on a sphere using the haversine formula.

    Parameters:
        lat1 (float): Latitude of the first point in decimal degrees.
        lon1 (float): Longitude of the first point in decimal degrees.
        lat2 (float): Latitude of the second point in decimal degrees.
        lon2 (float): Longitude of the second point in decimal degrees.

    Returns:
        float: Distance between the two points in kilometers.
    """
    # Earth's radius in kilometers (mean radius)
    R = 6371.0

    # Convert input coordinates from decimal degrees to radians
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    # Haversine formula components
    a = (
        math.sin(dphi / 2) ** 2
        + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    )
    # Central angle between the two points
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distance on the sphere’s surface
    return R * c

In [12]:
def compute_irregularity(coords: list) -> float:
    """
    Measure how “irregular” or meandering a path is by comparing its total length
    to the straight‑line distance between start and end points.

    Parameters:
        coords (List[Tuple[float, float]]): Ordered list of (lat, lon) pairs.
            Points with missing coordinates (None) are ignored.

    Returns:
        float: A value between 0 and 1, where 0 means the path is perfectly straight
               (direct distance == total distance) and values closer to 1 indicate
               more circuitous routes.
    """
    # 1. Filter out any points that have missing latitude or longitude
    v = [pt for pt in coords if None not in pt]
    # 2. If there are fewer than two valid points, no “path” exists, so irregularity is 0
    if len(v) < 2:
        return 0.0

    # 3. Compute the sum of distances between each pair of consecutive points
    total = sum(
        haversine(v[i][0], v[i][1], v[i+1][0], v[i+1][1])
        for i in range(len(v) - 1)
    )

    # 4. Compute the straight‑line (great‑circle) distance from the first to last point
    direct = haversine(v[0][0], v[0][1], v[-1][0], v[-1][1])

    # 5. If total path length is zero (all points identical), define irregularity as 0
    if total == 0.0:
        return 0.0

    # 6. Irregularity: proportion of “extra” distance beyond the direct line
    #    (higher means more deviation from straight line)
    return 1.0 - (direct / total)

In [13]:
def calculate_bearing(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """
    Calculate the initial bearing (forward azimuth) from point A to point B 
    on the Earth’s surface, using the haversine-based great-circle formula.

    Parameters:
        lat1 (float): Latitude of the start point in decimal degrees.
        lon1 (float): Longitude of the start point in decimal degrees.
        lat2 (float): Latitude of the end point in decimal degrees.
        lon2 (float): Longitude of the end point in decimal degrees.

    Returns:
        float: Bearing in degrees from north (0° ≤ bearing < 360°).
    """
    # Convert input coordinates from degrees to radians
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dlon = math.radians(lon2 - lon1)

    # Compute components for the atan2 function
    x = math.sin(dlon) * math.cos(phi2)
    y = (
        math.cos(phi1) * math.sin(phi2)
        - math.sin(phi1) * math.cos(phi2) * math.cos(dlon)
    )

    # atan2 returns the angle relative to the y-axis; convert to degrees and normalize
    initial_bearing = math.degrees(math.atan2(x, y))
    # Normalize bearing to 0–360° by adding 360 and taking modulo 360
    return (initial_bearing + 360) % 360

In [14]:
def compute_total_turning(coords: list) -> float:
    """
    Measure the total angular change (turning) along a path of coordinates.

    Parameters:
        coords (List[Tuple[float, float]]): Ordered list of (lat, lon) points.
            Points containing None are filtered out.

    Returns:
        float: Sum of absolute bearing changes (in degrees) between consecutive legs,
               normalized so that any turn >180° is taken the shorter way around.
               Returns 0.0 if fewer than three valid points are provided.
    """
    # 1. Filter out any points with missing latitude or longitude
    v = [pt for pt in coords if None not in pt]
    # 2. If fewer than 3 points, no meaningful "turn" can be computed
    if len(v) < 3:
        return 0.0

    # 3. Compute the bearing for each leg between consecutive points
    bears = [
        calculate_bearing(v[i][0], v[i][1], v[i+1][0], v[i+1][1])
        for i in range(len(v) - 1)
    ]

    total = 0.0
    # 4. For each consecutive pair of bearings, compute the smaller angular difference
    for i in range(len(bears) - 1):
        diff = abs(bears[i+1] - bears[i])
        # If the difference crosses the 0°/360° line, take the shorter path
        if diff > 180:
            diff = 360 - diff
        total += diff

    return total


In [15]:
def compute_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute path‑based flight metrics and append them to the DataFrame.

    Parameters:
        df (pd.DataFrame): Aggregated flight segments with a 'coordinates' column
                           (list of (lat, lon) tuples) for each segment.

    Returns:
        pd.DataFrame: The same DataFrame with two new columns:
            - 'irregularity': How much the path deviates from a straight line.
            - 'total_turning': Cumulative turning angle along the path.
    """
    # Calculate "irregularity" for each list of coordinates
    df['irregularity'] = df['coordinates'].apply(compute_irregularity)
    # Calculate total turning angle for each list of coordinates
    df['total_turning'] = df['coordinates'].apply(compute_total_turning)
    return df

## 5. Plotting the data

In [16]:
def build_geojson(df: pd.DataFrame) -> dict:
    """
    Convert a DataFrame of flight segments into a GeoJSON FeatureCollection.

    Parameters:
        df (pd.DataFrame): Must include columns:
            - 'coordinates': list of (lat, lon) tuples or a stringrepr.
            - 'flight':     flight identifier.
            - 'irregularity': float metric for detour index.
            - 'popup' (opt.): HTML content for map popups.

    Returns:
        dict: GeoJSON FeatureCollection with one LineString feature per row.
    """
    features = []

    # Iterate over each segment record
    for _, row in df.iterrows():
        coords = row['coordinates']
        # If stored as a string, parse safely into a Python list
        if isinstance(coords, str):
            coords = ast.literal_eval(coords)

        # Convert to GeoJSON [lon, lat] pairs, skipping None values
        coords_geo = [
            [lon, lat]
            for lat, lon in coords
            if lat is not None and lon is not None
        ]
        # Only include segments with at least two points
        if len(coords_geo) < 2:
            continue

        # Build the GeoJSON Feature
        features.append({
            'type': 'Feature',
            'geometry': {
                'type': 'LineString',
                'coordinates': coords_geo
            },
            'properties': {
                'flight':       row.get('flight', 'N/A'),
                'irregularity': float(row['irregularity']),
                'popup':        row.get('popup', '')
            }
        })

    # Wrap all features in a FeatureCollection
    return {
        'type': 'FeatureCollection',
        'features': features
    }

In [17]:
# 1. Establish SSH & SFTP connections for data retrieval.
ssh_client, sftp = get_sftp_client(read_config('config.yml'))
# The SSHClient from Paramiko provides an interface to connect to SSH servers.


In [18]:
# Widget & Control Definitions


# Date picker to choose which log date to fetch
date_picker = DatePicker(
    description="Log Date",
    value=datetime(2022, 10, 4).date()
)

# Slider to filter segments by minimum irregularity
irreg_slider = FloatSlider(
    description="Irregularity ≥",
    min=0.0, max=1.0, step=0.001,
    value=0.0,
    continuous_update=False,
    layout={"width": "80%"}
)

# Button to apply both filters
apply_btn = Button(
    description="Apply Filters",
    button_style="primary"
)

# Placeholder HTML for download link or messages
download_html = HTML("<em>No data yet</em>")

# Fullscreen overlay displayed during data loading
loading_html = HTML("""
<div style="
  position:fixed; top:0; left:0; width:100vw; height:100vh;
  background:rgba(255,255,255,0.8);
  display:flex; align-items:center; justify-content:center;
  z-index:10000;
">
  <div style="
    padding:20px 30px;
    background:white; border-radius:8px;
    box-shadow:0 2px 6px rgba(0,0,0,0.3);
    font-size:18px; font-weight:bold;
  ">Loading…</div>
</div>
""")

# Group widgets vertically and wrap in a map control
controls    = VBox([date_picker, irreg_slider, apply_btn, download_html])
widget_ctrl = WidgetControl(widget=controls, position="topright")
loading_ctrl= WidgetControl(widget=loading_html, position="topright")


/var/folders/jt/bxf8n9tx0jx2vwrv56lwq4c80000gp/T/ipykernel_89195/3298152229.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda grp: split_by_time_gap(grp, max_gap))
/var/folders/jt/bxf8n9tx0jx2vwrv56lwq4c80000gp/T/ipykernel_89195/3410488623.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series(aggregate_flight_segments(g)))


In [19]:
# Create the map, centered at (0,0) for initial view
m = Map(
    center=(0, 0),
    zoom=2,
    layout={'width':'100%', 'height':'700px'}  # Larger display area
)

# Empty GeoJSON layer; will hold flight segment lines
geo_layer = GeoJSON(
    data={'type':'FeatureCollection','features':[]},
    style={'color':'red', 'weight':3, 'opacity':0.8}
)

# Add the layer and control panel to the map
m.add_layer(geo_layer)
m.add_control(widget_ctrl)


In [20]:
# Hover Popup Handlers

# Track the current popup so we can remove it on hide
_popup = None

def _on_segment_hover(feature, **kwargs):
    """Show flight ID & irregularity when mousing over a segment."""
    global _popup
    # Remove previous popup if any
    if _popup:
        try: m.remove_layer(_popup)
        except: pass

    # Determine popup coordinates (event or centroid)
    coords = kwargs.get('coordinates')
    if coords:
        lon, lat = coords
    else:
        line = feature['geometry']['coordinates']
        lon = sum(pt[0] for pt in line) / len(line)
        lat = sum(pt[1] for pt in line) / len(line)

    # Build HTML content
    props = feature['properties']
    html = HTML(f"""
      <div style="
        font-size:12px; line-height:1.2em; padding:4px 6px;
        background:rgba(255,255,255,0.9); border-radius:4px;
        box-shadow:0 1px 3px rgba(0,0,0,0.2);
      ">
        <b>Flight:</b> {props['flight']}<br>
        <b>Irregularity:</b> {props['irregularity']:.3f}
      </div>
    """)

    # Create and add the popup to the map
    _popup = Popup(
        location=(lat, lon),
        child=html,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )
    m.add_layer(_popup)

def _on_segment_mouseout(feature, **kwargs):
    """Remove the popup when the mouse leaves the feature."""
    global _popup
    if _popup:
        try: m.remove_layer(_popup)
        except: pass
        _popup = None

# Attach handlers to the GeoJSON layer
geo_layer.on_hover(_on_segment_hover)
geo_layer.on_mouseout(_on_segment_mouseout)


In [21]:
# Data Pipeline Definition
# State variables to cache the last‐loaded date and DataFrames
_last_date = None
_df_cleaned = None
_df_metrics = None

def _run_pipeline(d):
    """
    Load and process the ADS‑B log for date `d`:
    1. Download & unzip JSON → DataFrame
    2. Clean timestamps & sort
    3. Segment by time gaps
    4. Aggregate per segment
    5. Compute irregularity & turning metrics
    """
    df_raw = load_and_clean_adsb_zip_json(sftp, build_remote_zip_path(d, LOG_PATH))
    df_c   = clean_adsb_data(df_raw)
    df_s   = segment_adsb(df_c, max_gap=10)
    df_a   = aggregate_segments(df_s)
    df_m   = compute_metrics(df_a)
    return df_c, df_m


In [22]:
def apply_all_filters(_=None):
    """
    Triggered by the Apply Filters button:
    - If the date changed, show loading overlay & reload pipeline
    - Filter by irregularity threshold
    - Update the GeoJSON layer and recenter the map
    - Generate a CSV download link
    """
    global _last_date, _df_cleaned, _df_metrics

    now = date_picker.value
    new_day = (now != _last_date)

    # Show loading overlay on new date
    if new_day:
        try: m.add_control(loading_ctrl)
        except ControlException: pass

    try:
        if new_day:
            _df_cleaned, _df_metrics = _run_pipeline(now)
            _last_date = now
            # Reset slider to data’s median
            mi = _df_metrics["irregularity"]
            irreg_slider.min = 0.0
            irreg_slider.max = float(mi.max())
            irreg_slider.value = float(mi.median())

        # Apply threshold filter
        thr = irreg_slider.value
        df_f = _df_metrics[_df_metrics["irregularity"] >= thr]

        # Update map layer & recenter
        geo_layer.data = build_geojson(df_f)
        m.center = (_df_cleaned["lat"].mean(), _df_cleaned["lon"].mean())
        m.zoom = 6

        # Create base64‑encoded CSV download link
        csv_bytes = df_f.to_csv(index=False).encode()
        b64 = base64.b64encode(csv_bytes).decode()
        download_html.value = (
            f'<a href="data:text/csv;base64,{b64}" '
            f'download="segments_{now}.csv">Download CSV</a>'
        )

    except FileNotFoundError:
        # No file for that date: clear layer & show error
        geo_layer.data = {'type':'FeatureCollection','features':[]}
        download_html.value = f"<span style='color:red;'>No log for {now}</span>"

    finally:
        # Hide loading overlay once done
        if new_day:
            try: m.remove_control(loading_ctrl)
            except ControlException: pass


In [23]:
# Wire the button click to our callback
apply_btn.on_click(apply_all_filters)

# Trigger initial draw on notebook load
apply_all_filters()

# Display the map in the notebook
from IPython.display import display
display(m)


/var/folders/jt/bxf8n9tx0jx2vwrv56lwq4c80000gp/T/ipykernel_89195/3298152229.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda grp: split_by_time_gap(grp, max_gap))
/var/folders/jt/bxf8n9tx0jx2vwrv56lwq4c80000gp/T/ipykernel_89195/3410488623.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series(aggregate_flight_segments(g)))


Map(center=[44.709366237447696, -74.8841393211297], controls=(ZoomControl(options=['position', 'zoom_in_text',…